## Demo notebook for fitting the model of _Brunton et. al. (2013)_

This is some boilerplate, for loading packages necessary for running the notebook

In [1]:
using Distributed
addprocs(4);

In [ ]:
using PyPlot, JLD2, DataFrames
using LinearAlgebra, Pandas, GLM

┌ Info: Precompiling PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee]
└ @ Base loading.jl:1192
┌ Info: Installing matplotlib via the Conda matplotlib package...
└ @ PyCall /Users/briandepasquale/.julia/packages/PyCall/ttONZ/src/PyCall.jl:705
┌ Info: Running `conda install -y matplotlib` in root environment
└ @ Conda /Users/briandepasquale/.julia/packages/Conda/CpuvI/src/Conda.jl:112


Solving environment: ...working... done

## Package Plan ##

  environment location: /Users/briandepasquale/.julia/conda/3

  added / updated specs:
    - matplotlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cycler-0.10.0              |           py37_0          14 KB
    freetype-2.9.1             |       hb4e5f40_0         864 KB
    kiwisolver-1.0.1           |   py37h0a44026_0          56 KB
    libpng-1.6.37              |       ha441bb4_0         325 KB
    matplotlib-3.0.3           |   py37h54f8f79_0         6.6 MB
    pyparsing-2.4.0            |             py_0          58 KB
    python-dateutil-2.8.0      |           py37_0         281 KB
    pytz-2019.1                |             py_0         236 KB
    tornado-6.0.2              |   py37h1de35cc_0         642 KB
    ------------------------------------------------------------
                                      

In [3]:
using pulse_input_DDM

┌ Info: Recompiling stale cache file /Users/briandepasquale/.julia/compiled/v1.1/pulse_input_DDM/m8WoA.ji for pulse_input_DDM [c2542da2-1f7d-11e9-2506-87691095ebb2]
└ @ Base loading.jl:1184
┌ Warning: Module StatsBase with build ID 227921036430982 is missing from the cache.
│ This may mean StatsBase [2913bbd2-ae8a-5f71-8c99-4fb6c76f3a91] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:947
┌ Info: Recompiling stale cache file /Users/briandepasquale/.julia/compiled/v1.1/Optim/R5uoh.ji for Optim [429524aa-4258-5aef-a3af-852621145aeb]
└ @ Base loading.jl:1184
┌ Warning: Module StatsBase with build ID 227921036430982 is missing from the cache.
│ This may mean StatsBase [2913bbd2-ae8a-5f71-8c99-4fb6c76f3a91] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:947
┌ Info: Recompiling stale cache file /Users/briandepasquale/.julia/compiled/v1.1/JLD/l8ijy.ji for JLD [4138dd39-2aa7-5051-a626-17a0bb65d9c8]
└ @ Base 

      From worker 5:	┌ Warning: Module DataFrames with build ID 227942982430769 is missing from the cache.
      From worker 5:	│ This may mean DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0] does not support precompilation but is imported by a module that does.
      From worker 5:	└ @ Base loading.jl:947
      From worker 4:	┌ Warning: Module DataFrames with build ID 227942982402836 is missing from the cache.
      From worker 4:	│ This may mean DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0] does not support precompilation but is imported by a module that does.
      From worker 4:	└ @ Base loading.jl:947
      From worker 2:	┌ Warning: Module DataFrames with build ID 227939112442822 is missing from the cache.
      From worker 2:	│ This may mean DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0] does not support precompilation but is imported by a module that does.
      From worker 2:	└ @ Base loading.jl:947
      From worker 4:	┌ Warning: Module DataFrames with build ID 227942

┌ Warning: The call to compilecache failed to create a usable precompiled cache file for pulse_input_DDM [c2542da2-1f7d-11e9-2506-87691095ebb2]
│   exception = Required dependency ROCAnalysis [f535d66d-59bb-5153-8d2b-ef0a426c6aff] failed to load from a cache file.
└ @ Base loading.jl:969


Define dictionaries with some generative parameters, a Boolean vector of which ones to fit, and initial values.

In [4]:
#parameters of the latent model
pz_gen = Dict("generative" => vcat(1.,13.,-0.5,10.,1.,0.4,0.02), 
    "name" => vcat("σ_i","B", "λ", "σ_a","σ_s","ϕ","τ_ϕ"),
    "fit" => vcat(trues(7)),
    "initial" => vcat(2.,15.,-5.,100.,2.,0.2,0.005),
    "lb" => [eps(), 4., -5., eps(), eps(), eps(), eps()],
    "ub" => [10., 100, 5., 800., 40., 2., 10.])

#parameters for the choice observation
pd_gen = Dict("generative" => vcat(0.1,0.1), 
    "name" => vcat("bias","lapse"), "fit" => trues(2), 
    "initial" => vcat(0.,0.5));

## simulate choices from a model, to check that generative parameters can be found

In [5]:
#define how many trials to generate data from and the time binning scale (in seconds)
ntrials,dt = Int(0.1e4), 1e-2

(1000, 0.01)

In [6]:
#generate some simulated clicks times and trial durations
data = pulse_input_DDM.sample_clicks(ntrials,dt);

In [7]:
#simulate choices from the model, given the generative parameters
pulse_input_DDM.sampled_dataset!(data, pz_gen["generative"], pd_gen["generative"]; num_reps=1, rng=4);

RemoteException: On worker 2:
UndefVarError: ##196#197 not defined
deserialize_datatype at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Serialization/src/Serialization.jl:1115
handle_deserialize at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Serialization/src/Serialization.jl:771
deserialize at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Serialization/src/Serialization.jl:731
deserialize_datatype at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Serialization/src/Serialization.jl:1139
handle_deserialize at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Serialization/src/Serialization.jl:771
deserialize at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Serialization/src/Serialization.jl:731
handle_deserialize at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Serialization/src/Serialization.jl:

In [ ]:
#compute the likelihood of the data, given the generative parameters
compute_LL(pz_gen["generative"], pd_gen["generative"],data)

In [ ]:
#find the ML parameters with gradient descent
@time pz_gen, pd_gen, = optimize_model(pz_gen, pd_gen, data)

In [ ]:
#compute the Hessian of the LL landscape, to compute confidence intervals on the parameters
compute_H_CI(pz_gen, pd_gen, data, dt)

In [ ]:
#identify which ML parameters have generative parameters within the CI 
pz_gen[:within_bounds] = (pz_gen[:CI_minus] .< pz_gen[:generative]) .& (pz_gen[:CI_plus] .> pz_gen[:generative])
pd_gen[:within_bounds] = (pd_gen[:CI_minus] .< pd_gen[:generative]) .& (pd_gen[:CI_plus] .> pd_gen[:generative]);

In [ ]:
DataFrames.DataFrame(pz_gen)

In [ ]:
pd_gen

In [ ]:
#sample data from the model using the ML parameters
sampled_choices = pmap((T,L,R) -> sample_model(T,L,R,pz_gen[:final],pd_gen[:final]),
    data["T"],data["leftbups"],data["rightbups"])

#compute the final click difference, which will dictate the correct choice
ΔLR = map((nT,L,R)->diffLR(nT,L,R,dt),data["nT"],data["leftbups"],data["rightbups"])
ΔLR = map(x-> x[end], ΔLR);

In [ ]:
#bin the real choices and the sampled choices
nbins = 15;
conds,qcut_bins = qcut(ΔLR,nbins,labels=false,retbins=true);
conds = conds .+ 1;

frac_choice_sampled = fill!(Vector{Float64}(undef,nbins),NaN)
frac_choice = fill!(Vector{Float64}(undef,nbins),NaN)
for i = 1:nbins
    frac_choice_sampled[i] = mean(sampled_choices[conds .== i])
    frac_choice[i] = mean(data["pokedR"][conds .== i])
end

In [ ]:
#fit a GLM to the real choices and the simulated choices
GLM_dataframe = DataFrames.DataFrame(X= ΔLR, Y = data["pokedR"])
GLM_data = glm(@formula(Y ~ X), GLM_dataframe, Binomial(), LogitLink())
GLM_samplesframe = DataFrames.DataFrame(X= ΔLR, Y = sampled_choices)
GLM_samples = glm(@formula(Y ~ X), GLM_samplesframe, Binomial(), LogitLink())

In [ ]:
fig = figure(figsize=(6,6))
ax = subplot(111)

scatter(qcut_bins[1:end-1] + diff(qcut_bins)/2, frac_choice, color="red", label="data")
scatter(qcut_bins[1:end-1] + diff(qcut_bins)/2, frac_choice_sampled, color="grey", label="sampled")

PyPlot.plot(sort(GLM_dataframe[:X]), sort(predict(GLM_data)), color="red")
PyPlot.plot(sort(GLM_samplesframe[:X]), sort(predict(GLM_samples)), color="grey")

ylabel("% poked R")
xlabel(L"\Delta{LR}")
ax[:spines]["top"][:set_color]("none") # Remove the top axis boundary
ax[:spines]["right"][:set_color]("none") # Remove the top axis boundary
legend()

## fit real data

In [ ]:
path = "/mnt/bucket/labs/brody/vdtang/P055_processed"
ratnames=["P055"]
sessids=[[17061800,18041700,16052600,16070500,16111700,18010300,18022600,17122200,
        17081200,17092600,17051900,18030500,17011800,18012200,17011500,17062500,
        17112700,18032700,16082200,18060800,18032500,17082900,17080500,17081000,
        17052300,17090800,17090900,18041500,18040500,17122000]];

In [ ]:
data = aggregate_choice_data(path, sessids, ratnames, dt)

## saving things

In [ ]:
JLD2.@save "/usr/people/briandd/Projects/pulse_input_DDM.jl/data/results/marino.jld" pz pd pzF pdF